In [66]:
import re
import json
import thulac
import pandas as pd
from tqdm.notebook import tqdm

thu = thulac.thulac(seg_only=True, user_dict="./text/TextDict.txt", deli=" ")

# thu.cut("", text=True)

Model loaded succeed


In [57]:
def cleanText(sentence):
    # punctuation = set(["。", "，", "？", "！", "“", "”", "（", "）"])
    # cleaned = ''.join([char for char in cleaned if char not in punctuation])
    cleaned = re.sub(r'\[.*?\]', "", sentence)

    return cleaned

defined = []
with open("./text/TextDict.txt", encoding="utf-8") as f:
    for line in f:
        defined.append(line.strip().split(" ")[0])
    
pattern = f"({'|'.join(defined)})"  # make sure to leave the delimiters in

def preProcess(sentence: str):
    cleaned = sentence.lower()
    cleaned = re.sub(r'\[.*?\]', "", cleaned)  # removes emotes
    cleaned.replace("\n", "")

    # removes japanese characters
    jpPattern = r'[\u3040-\u309F\u30A0-\u30FF\uFF65-\uFF9F]'
    cleaned = re.sub(jpPattern, "", cleaned)
    
    parts = re.split(pattern, cleaned)
    newSentence = ' '.join([part for part in parts if part])

    return newSentence

In [63]:
from collections import Counter

mainCounter = Counter()

for i in tqdm(range(1, 54)):
    with open(f"./data/ba-mahjong/DataFile{i}.json", encoding="utf-8") as f:
        data = json.load(f)
    df = pd.DataFrame(data)

    for j in range(len(df['content'])):
        current = preProcess(df['content'][j])
        words = [l[0] for l in thu.cut(current)]

        mainCounter += Counter(words)

  0%|          | 0/53 [00:00<?, ?it/s]

In [64]:
with open("./text/Counterv2.1.json", "w+", encoding="utf-8") as f:
    json.dump(dict(sorted(mainCounter.items(), key=lambda item:item[1], reverse=True)), f, ensure_ascii=False)